In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import ClassificationModel, EmbeddingsModel
from trainer import SiameseTrainerCombinedLoss, all_subjects
import torch

import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [2]:
subjects_test = [all_subjects[0]]
subjects_train = [sub for sub in all_subjects if sub not in subjects_test]

In [3]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "lambda_loss": {'max': 0.5, 'min': 0.01}
    }
}

In [4]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test,
            "dataset_ignore_sample_subject": True,
            "wandb": True,
            "log": False,
            "filter": [],
            "batch_size": 128,
            "batch_size_test": 128,
            "margin": 0.04,
            "weight_decay": 0.0001,
            "learning_rate": 0.001,
            "lr_steps": 20,
            "lambda_loss": config.lambda_loss,
            "number_steps": 200,
            "number_steps_testing": 20
        }

        model_embedding_hyperparameter = {
            "sigmoid": False,
            "dropout": 0.3,
            "layers": [207, 512, 64]
        }

        model_classifier_hyperparameter = {
            "dropout": 0.3,
            "layers": [64, 32]
        }

        classifier_model = ClassificationModel(model_classifier_hyperparameter)
        embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
        _ = embedding_model.load_state_dict(torch.load("D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Results\models\embedding_models\model_abs_acc_89.40.pth"))
        #filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
        trainer = SiameseTrainerCombinedLoss(trainer_hyperparameter, classifier_model, embedding_model, filter=None)

        trainer.trainloop(10)

In [5]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection_Combined_Loss_SiameseNetwork")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 5rs3u6c3
Sweep URL: https://wandb.ai/svenk/PainLevelShiftDetection_Combined_Loss_SiameseNetwork/sweeps/5rs3u6c3


In [6]:
wandb.agent(sweep_id, train, count=30)

wandb: Agent Starting Run: eo8wvxth with config:
wandb: 	lambda_loss: 0.3130359909338497
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


accuracy,▁▃▃▆▅█▅▅██
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.54167
epoch,10


wandb: Agent Starting Run: 50ox89nt with config:
wandb: 	lambda_loss: 0.40154774056675224
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


accuracy,▁▃▂▂▂█▄▅▄▃
epoch,▁▂▃▃▄▅▆▆▇█
accuracy,0.46726
epoch,10


wandb: Agent Starting Run: zpissl05 with config:
wandb: 	lambda_loss: 0.2597638880895929
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
